<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/Constructing_Historic_Median_Sales_Price_per_Postal_Area_and_per_bedroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

14.07.21
Identifying PostCode areas in the property sales dataset (6 months - Jan, Feb, Mar, Apr, May, June 2018) and obtaining Median Sales Price per bedroom per PostCode area.



In [1]:
!pip install harperdb

In [2]:
import harperdb
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
url='https://ce901data-orc0770.harperdbcloud.com'
db=harperdb.HarperDB(url=url,
            username='myuser',
            password='abcdef',
            timeout=500)

In [4]:
db

Below data is extracted from the SQL database, however because of poor performance of the JOIN SQL operations, instead of relational SQL statements, the JOIN operations are performed in pandas DataFrames (according to https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).


The data extraction from SQL tables in placed within the loop to avoid the database timeout issues.


In [5]:
list_of_dfs = {}
count=0
for mt in range(1, 7):
  for yr in range (2018,2019):
    year=yr
    month=mt
    count=count+1
    #sqlstring1='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date '
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE (num_of_beds>0 and deed_date LIKE "%{}/{}")'
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE deed_date LIKE "%{}/{}"'
    #sqlstring=sqlstring1+sqlstring2
    sqlstring='SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%{}/{}"'
    sqlstring=sqlstring.format(month,year)
    print(sqlstring)
    my_db_data=db.sql(sqlstring)
    list_of_dfs[count] = pd.DataFrame(my_db_data)
    print(list_of_dfs[count].shape)
df_prop_sales = pd.concat(list_of_dfs, sort=False)
#excluding 'other' property types (i.e. commercial) from the properties sales data 
df_prop_sales=df_prop_sales[(df_prop_sales['property_type']!='O')]
df_prop_sales

SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%1/2018"
(169065, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%2/2018"
(157760, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%3/2018"
(84147, 8)
SELECT propertypurchases.unique_id, propertypurchases

unique_id  price_paid  ...   deed_date  postcode
1 0      64342BFD-ABA7-422C-E053-6C04A8C0FB8A     1740000  ...  11/01/2018    E8 4NE
  1      64342BFD-ABAA-422C-E053-6C04A8C0FB8A     1525000  ...  19/01/2018    N5 1BZ
  2      64342BFD-ABAB-422C-E053-6C04A8C0FB8A     3306000  ...  17/01/2018   W11 1NZ
  3      64342BFD-ABAC-422C-E053-6C04A8C0FB8A      442000  ...  04/01/2018   IG3 9NB
  4      64342BFD-ABB0-422C-E053-6C04A8C0FB8A     1375000  ...  12/01/2018   W12 8JN
...                                       ...         ...  ...         ...       ...
6 95756  C18F412A-A064-81A6-E053-6B04A8C0AD18       75995  ...  29/06/2018  NP19 9PX
  95757  C18F412B-0753-81A6-E053-6B04A8C0AD18      595000  ...  29/06/2018  LE14 3PT
  95758  C18F412B-097B-81A6-E053-6B04A8C0AD18      181967  ...  29/06/2018   LE8 0XG
  95759  C18F412B-3469-81A6-E053-6B04A8C0AD18      510000  ...  07/06/2018   OX9 3GF
  95760  C18F412B-6598-81A6-E053-6B04A8C0AD18      485000  ...  12/06/2018  ME19 5SQ

[618873 rows x 8 columns]

In [6]:
sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds FROM projectdata.bedrooms WHERE num_of_beds>0'
bedrooms=db.sql(sqlstring)
df_bedrooms=pd.DataFrame(bedrooms)
df_bedrooms.rename(columns = {'id':'unique_id'}, inplace = True)
df_bedrooms

,unique_id,num_of_beds
0,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1
1,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1
2,6B32222C-3F37-01F1-E053-6C04A8C0D2C8,1
3,68FEB20C-085C-38DA-E053-6C04A8C051AE,1
4,6DA08449-D320-30F2-E053-6B04A8C05F3B,1
...,...,...
258369,6B32222C-F1EF-01F1-E053-6C04A8C0D2C8,16
258370,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17
258371,64342BFF-158B-422C-E053-6C04A8C0FB8A,23
258372,75050A86-10FB-9A88-E053-6B04A8C02390,46


In [7]:
#sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE num_of_beds>0'
#my_db_data=db.sql(sqlstring)
#data=pd.DataFrame(my_db_data)
data = pd.merge(df_bedrooms, df_prop_sales, how="left", on="unique_id")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
1,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH
2,6B32222C-3F37-01F1-E053-6C04A8C0D2C8,1,175000.0,HATFIELD,WELWYN HATFIELD,L,F,20/03/2018,AL10 8UP
3,68FEB20C-085C-38DA-E053-6C04A8C051AE,1,227000.0,LONDON,GREENWICH,L,F,26/02/2018,SE9 2LL
4,6DA08449-D320-30F2-E053-6B04A8C05F3B,1,125000.0,EASTBOURNE,EASTBOURNE,L,F,25/04/2018,BN22 7PN
...,...,...,...,...,...,...,...,...,...
258369,6B32222C-F1EF-01F1-E053-6C04A8C0D2C8,16,695000.0,HASTINGS,HASTINGS,F,S,12/03/2018,TN34 1SJ
258370,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA
258371,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA
258372,75050A86-10FB-9A88-E053-6B04A8C02390,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#check rows where postcode is NAN - these are the result of rows added on the Join with bedrooms data where bedrooms data is for sale happened after the considered period
#data_t=data[ (data['postcode'].isnull()) ]
#drop NAN rows from data
data=data.dropna(subset=['postcode'])     
data 

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
1,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH
2,6B32222C-3F37-01F1-E053-6C04A8C0D2C8,1,175000.0,HATFIELD,WELWYN HATFIELD,L,F,20/03/2018,AL10 8UP
3,68FEB20C-085C-38DA-E053-6C04A8C051AE,1,227000.0,LONDON,GREENWICH,L,F,26/02/2018,SE9 2LL
4,6DA08449-D320-30F2-E053-6B04A8C05F3B,1,125000.0,EASTBOURNE,EASTBOURNE,L,F,25/04/2018,BN22 7PN
...,...,...,...,...,...,...,...,...,...
258367,666758D7-2CD3-3363-E053-6B04A8C0D74E,13,840000.0,HARROGATE,HARROGATE,F,D,21/02/2018,HG1 2HS
258369,6B32222C-F1EF-01F1-E053-6C04A8C0D2C8,16,695000.0,HASTINGS,HASTINGS,F,S,12/03/2018,TN34 1SJ
258370,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA
258371,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA


In [98]:
#postcode area with month and year
data_prep=data
data_prep['postcode_area_m_y']=data_prep['postcode'].str.split(' ').str[0]+'_'+data_prep['deed_date'].str.split('/').str[1]+'_'+data_prep['deed_date'].str.split('/').str[2]
data_prep['price_per_bedroom']=data_prep['price_paid']/data_prep['num_of_beds']
data_prep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom
0,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000
1,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH,CB9_03_2018,156500.000000
2,6B32222C-3F37-01F1-E053-6C04A8C0D2C8,1,175000.0,HATFIELD,WELWYN HATFIELD,L,F,20/03/2018,AL10 8UP,AL10_03_2018,175000.000000
3,68FEB20C-085C-38DA-E053-6C04A8C051AE,1,227000.0,LONDON,GREENWICH,L,F,26/02/2018,SE9 2LL,SE9_02_2018,227000.000000
4,6DA08449-D320-30F2-E053-6B04A8C05F3B,1,125000.0,EASTBOURNE,EASTBOURNE,L,F,25/04/2018,BN22 7PN,BN22_04_2018,125000.000000
...,...,...,...,...,...,...,...,...,...,...,...
258367,666758D7-2CD3-3363-E053-6B04A8C0D74E,13,840000.0,HARROGATE,HARROGATE,F,D,21/02/2018,HG1 2HS,HG1_02_2018,64615.384615
258369,6B32222C-F1EF-01F1-E053-6C04A8C0D2C8,16,695000.0,HASTINGS,HASTINGS,F,S,12/03/2018,TN34 1SJ,TN34_03_2018,43437.500000
258370,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA,NP4_01_2018,8235.294118
258371,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA,WV2_01_2018,3478.260870


In [63]:
def setThreeMonthsPlus(anId,aMonth,aYear):
  if aMonth>=10:
    tMonth=aMonth+3-12
    tYear=aYear+1
  else:
    tMonth=aMonth+3
    tYear=aYear
  myNewId= str(anId).split('_')[0]+'_'+str(tMonth).zfill(2) +'_'+str(tYear)
  return myNewId

In [103]:
g_median_price_per_bed = data_prep.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
g_median_price_per_bed['m_y']=g_median_price_per_bed['postcode_area_m_y'].str[-7:]
g_median_price_per_bed

,postcode_area_m_y,price_per_bedroom,m_y
0,AL10_01_2018,117916.666667,01_2018
1,AL10_02_2018,121666.666667,02_2018
2,AL10_03_2018,118666.666667,03_2018
3,AL10_04_2018,131666.666667,04_2018
4,AL10_05_2018,122500.000000,05_2018
...,...,...,...
12132,YO7_04_2018,75000.000000,04_2018
12133,YO8_01_2018,63500.000000,01_2018
12134,YO8_02_2018,61000.000000,02_2018
12135,YO8_03_2018,55333.333333,03_2018


In [104]:
#Need to add the following assumption: if data belongs to 1/2018, 2/2018 and 3/2018 - we do not have historic correspondong data for 10/2017, 11/2017 and 12/2017,
#so the working assumption that median prices per bedroom in 10/2017 were the same as in 1/2018, median prices per bedroom in 11/2017 were the same as in 2/2018, median prices per bedroom in 12/2017 were the same as in 3/2018.
#To implement this assumption - we'll be getting the section of the above table for  1/2018, 2/2018 and 3/2018 so this will be appended to the new table as price_per_bedroom_3mnths_ago
df_add_assumption=g_median_price_per_bed[(g_median_price_per_bed['m_y']=='01_2018')|(g_median_price_per_bed['m_y']=='02_2018')|(g_median_price_per_bed['m_y']=='03_2018')]
df_add_assumption.rename(columns = {'price_per_bedroom':'median_ppb_3mnths_ago'}, inplace = True)
df_add_assumption=df_add_assumption.drop(['m_y'], axis = 1)
df_add_assumption

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_01_2018,117916.666667
1,AL10_02_2018,121666.666667
2,AL10_03_2018,118666.666667
6,AL1_01_2018,189083.333333
7,AL1_02_2018,183202.500000
...,...,...
12130,YO7_02_2018,67333.333333
12131,YO7_03_2018,55000.000000
12133,YO8_01_2018,63500.000000
12134,YO8_02_2018,61000.000000


In [92]:
list_of_months_plus_three=[]
for row in g_median_price_per_bed.itertuples():
   myMonth=int(str(row[1]).split('_')[1])
   myYear =int((str(row[1])).split('_')[2])
   threeMnthsPlus=setThreeMonthsPlus(row[1],myMonth,myYear)
   new_row=[]
   new_row.append(threeMnthsPlus)
   new_row.append(row[2])
   list_of_months_plus_three.append(new_row)


In [105]:
g_median_price_per_bed = pd.DataFrame(list_of_months_plus_three)
g_median_price_per_bed.columns =[ 'postcode_area_m_y','median_ppb_3mnths_ago', ]
g_median_price_per_bed

,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_04_2018,117916.666667
1,AL10_05_2018,121666.666667
2,AL10_06_2018,118666.666667
3,AL10_07_2018,131666.666667
4,AL10_08_2018,122500.000000
...,...,...
12132,YO7_07_2018,75000.000000
12133,YO8_04_2018,63500.000000
12134,YO8_05_2018,61000.000000
12135,YO8_06_2018,55333.333333


In [106]:
g_median_price_per_bed_3m_ago = pd.concat([df_add_assumption, g_median_price_per_bed], axis=0)
g_median_price_per_bed_3m_ago

,postcode_area_m_y,median_ppb_3mnths_ago
0,AL10_01_2018,117916.666667
1,AL10_02_2018,121666.666667
2,AL10_03_2018,118666.666667
6,AL1_01_2018,189083.333333
7,AL1_02_2018,183202.500000
...,...,...
12132,YO7_07_2018,75000.000000
12133,YO8_04_2018,63500.000000
12134,YO8_05_2018,61000.000000
12135,YO8_06_2018,55333.333333


In [107]:
data_prep = pd.merge(data_prep, g_median_price_per_bed_3m_ago, how="left", on="postcode_area_m_y")
data_prep

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,price_per_bedroom_3mnths_ago,median_ppb_3mnths_ago
0,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000,73416.666667,73416.666667
1,6B32222C-9C99-01F1-E053-6C04A8C0D2C8,1,156500.0,HAVERHILL,ST EDMUNDSBURY,F,T,29/03/2018,CB9 0EH,CB9_03_2018,156500.000000,91666.666667,91666.666667
2,6B32222C-3F37-01F1-E053-6C04A8C0D2C8,1,175000.0,HATFIELD,WELWYN HATFIELD,L,F,20/03/2018,AL10 8UP,AL10_03_2018,175000.000000,118666.666667,118666.666667
3,68FEB20C-085C-38DA-E053-6C04A8C051AE,1,227000.0,LONDON,GREENWICH,L,F,26/02/2018,SE9 2LL,SE9_02_2018,227000.000000,141666.666667,141666.666667
4,6DA08449-D320-30F2-E053-6B04A8C05F3B,1,125000.0,EASTBOURNE,EASTBOURNE,L,F,25/04/2018,BN22 7PN,BN22_04_2018,125000.000000,76666.666667,76666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256257,666758D7-2CD3-3363-E053-6B04A8C0D74E,13,840000.0,HARROGATE,HARROGATE,F,D,21/02/2018,HG1 2HS,HG1_02_2018,64615.384615,90500.000000,90500.000000
256258,6B32222C-F1EF-01F1-E053-6C04A8C0D2C8,16,695000.0,HASTINGS,HASTINGS,F,S,12/03/2018,TN34 1SJ,TN34_03_2018,43437.500000,95125.000000,95125.000000
256259,6B32222C-0F70-01F1-E053-6C04A8C0D2C8,17,140000.0,PONTYPOOL,TORFAEN,F,T,05/01/2018,NP4 6DA,NP4_01_2018,8235.294118,36008.333333,36008.333333
256260,64342BFF-158B-422C-E053-6C04A8C0FB8A,23,80000.0,WOLVERHAMPTON,WOLVERHAMPTON,F,T,12/01/2018,WV2 4NA,WV2_01_2018,3478.260870,36666.666667,36666.666667


In [109]:
data_test = data_prep[(data_prep['median_ppb_3mnths_ago'].isnull())]
data_test

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,price_per_bedroom_3mnths_ago,median_ppb_3mnths_ago
286,726BF13B-4FD8-0A46-E053-6C04A8C01D0D,1,112500.0,SHEFFIELD,SHEFFIELD,L,T,01/05/2018,S14 1BZ,S14_05_2018,112500.000000,NaN,NaN
565,773788C2-F22F-2CE4-E053-6C04A8C05E57,1,48000.0,KING'S LYNN,KING'S LYNN AND WEST NORFOLK,L,F,27/06/2018,PE30 1NH,PE30_06_2018,48000.000000,NaN,NaN
635,7011B109-14D5-8ED6-E053-6B04A8C075C1,1,60000.0,PETERBOROUGH,FENLAND,L,F,04/06/2018,PE7 1BD,PE7_06_2018,60000.000000,NaN,NaN
679,7E86B6FA-82AB-458C-E053-6B04A8C0C84C,1,130000.0,PETERBOROUGH,HUNTINGDONSHIRE,F,D,04/06/2018,PE7 3YR,PE7_06_2018,130000.000000,NaN,NaN
761,7011B10A-21A5-8ED6-E053-6B04A8C075C1,1,98000.0,PETERBOROUGH,CITY OF PETERBOROUGH,F,T,13/06/2018,PE4 5AG,PE4_06_2018,98000.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255955,726BF13A-F298-0A46-E053-6C04A8C01D0D,7,300000.0,BOSTON,BOSTON,F,D,26/06/2018,PE20 1BX,PE20_06_2018,42857.142857,NaN,NaN
256036,7011B109-2DB8-8ED6-E053-6B04A8C075C1,7,243000.0,PLYMOUTH,CITY OF PLYMOUTH,F,S,01/06/2018,PL7 4BJ,PL7_06_2018,34714.285714,NaN,NaN
256079,7011B10A-149C-8ED6-E053-6B04A8C075C1,8,455000.0,LEEDS,LEEDS,F,D,04/06/2018,LS2 9PR,LS2_06_2018,56875.000000,NaN,NaN
256170,726BF13A-8D76-0A46-E053-6C04A8C01D0D,8,310000.0,LIFTON,WEST DEVON,F,D,29/06/2018,PL16 0AN,PL16_06_2018,38750.000000,NaN,NaN


In [110]:
g_test = data_test.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
g_test


,postcode_area_m_y,price_per_bedroom
0,B19_06_2018,4.250000e+04
1,BS29_06_2018,8.795833e+04
2,CA21_05_2018,3.650000e+04
3,CA25_05_2018,2.800000e+04
4,CA27_04_2018,7.000000e+04
...,...,...
185,W1T_04_2018,5.237500e+05
186,WC1X_04_2018,6.163013e+05
187,WC2N_04_2018,1.012500e+06
188,WV15_04_2018,5.558333e+04
